In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Pre-trained ResNet50 model without top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)  # Adjust the number of classes

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# train_data_dir = 'C:/Users/ITBS/desktop/disease_dataset/train'
# test_data_dir = 'C:/Users/ITBS/desktop/disease_dataset/test'


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 23s 0us/step


In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
img_size = (224, 224)
batch_size = 32

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Normalization for validation/testing
test_datagen = ImageDataGenerator(rescale=1./255)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    'C:/Users/ITBS/desktop/disease_dataset/train',  # Replace with your actual path
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Testing data generator
test_generator = test_datagen.flow_from_directory(
    'C:/Users/ITBS/desktop/disease_dataset/test',  # Replace with your actual path
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 14558 images belonging to 20 classes.
Found 2637 images belonging to 20 classes.


In [11]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load pre-trained ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(20, activation='softmax')(x)  # Adjust for 20 classes

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)


In [12]:
from tensorflow.keras.optimizers import Adam

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)


In [15]:
# Save the trained model
model.save('C:/Users/ITBS/desktop/disease_detection.keras')  # Replace with your desired save path

In [3]:
import numpy as np
from PIL import Image
import tensorflow as tf

# Load the trained disease detection model
disease_detection_model = tf.keras.models.load_model(r'C:/Users/ITBS/Desktop/disease_detection.keras')

def run_model(img_path):
    def preprocess_image(image_path, target_size):
        img = Image.open(image_path).resize(target_size)  # Resize to match model's input shape
        img_array = np.array(img).astype('float32') / 255.0  # Normalize to [0, 1] range
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        return img_array

    # Define target size for the disease detection model
    disease_detection_target_size = (224, 224)  # Resize to match model's input shape

    # Preprocess the input image for disease detection
    img_array_for_disease = preprocess_image(img_path, disease_detection_target_size)

    # Predict the disease type
    try:
        disease_predictions = disease_detection_model.predict(img_array_for_disease)
        disease_class_index = np.argmax(disease_predictions, axis=1)[0]

        # List of disease class names (replace with actual disease names)
        disease_class_names = ['Corn_blight', 'Corn_common_rust', 'Corn_gray_leaf_spot', 'Corn_healthy', 
                            'Potato_Early_Blight', 'Potato_Healthy', 'Potato_Late_Blight', 
                            'Rice_BrownSpot', 'Rice_Healthy', 'Rice_Hispa', 'Rice_LeafBlast', 
                            'Tomato_bacterial_spot', 'Tomato_early_blight', 'Tomato_healthy', 
                            'Tomato_late_blight', 'Tomato_leaf_miner', 'Tomato_spotted_wilt_virus', 
                            'Wheat_Brown_Rust', 'Wheat_Healthy', 'Wheat_Yellow_Rust']
        predicted_disease = disease_class_names[disease_class_index]

        print(f'Predicted disease: {predicted_disease}')
    except Exception as e:
        print(f'Error during disease prediction: {e}')

run_model(img_path = r'C:/Users/ITBS/Desktop/rlb.jpg')

C:\Users\ITBS\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 218 variables whereas the saved optimizer has 434 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted disease: Rice_LeafBlast
